In [ ]:
# vytvorim si ndarray a dataset kluce budu nazov adresara a kod speakra a hodnota bude nazov suboru 
# potom tomu priradim oznacenie train a testovacej sady tak aby kazdy bol aspon raz v testovacej 
# a zaroven sa nevyskytoval v treningovej 
import pandas as pd
import numpy as np
import os
path="./train/audio/"
labels=os.listdir(path)
file_list=[]
user_id_list=[]
word_list=[]
for file in labels:
    file_list.append(os.listdir(path+file))
    user_id_list.append([x.split(sep="_")[0] for x in file_list[-1]])
    word_list.append([file for x in file_list[-1]])

In [ ]:
metadata=pd.DataFrame({'word':np.concatenate(word_list,axis=0),
              'user_id':np.concatenate(user_id_list,axis=0),
              'file':np.concatenate(file_list,axis=0)})
# metadata.set_index(["word","user_id"],inplace=True)
pocty = metadata.groupby(['user_id']).count()
pocty['fold']=0
pocty.loc[0:360,'fold']=0
pocty.loc[361:730,'fold']=1
pocty.loc[731:1080,'fold']=2
pocty.loc[1081:1420,'fold']=3
pocty.loc[1421:,'fold']=4
metadata = metadata.merge(pocty['fold'],on='user_id')

In [ ]:
import librosa
def prepare_dataset(path,metadata):
    vectors = []
    for i in metadata.index.tolist():
        fldr = metadata.loc[i,'word']
        file_name = metadata.loc[i,'file']
        wave, sr = librosa.load(path+fldr+'/'+file_name, mono=True, sr=None)
        # Downsampling
        wave = wave[::3]
        mfcc = librosa.feature.mfcc(wave, sr=16000)
        vectors.append(mfcc)
    return vectors

m_data = metadata[metadata['word']!='_background_noise_']

# vynecham speakrov iba s jednym slovom 
pocet_slov = m_data[['user_id','word']].groupby('user_id').nunique()
zoznam = pocet_slov[pocet_slov['word']<2].index
m_data = m_data[~m_data.user_id.isin(zoznam)]
m_data.reset_index(drop=True,inplace=True)

data = prepare_dataset(path=path,metadata=m_data)

In [ ]:
m_data

In [ ]:
import tensorflow as tf
import random
import numpy as np

def sub_vec(vec,limit=4):
    x = np.array(vec)
    x = x.T
    k = 0
    if len(x)-limit>0:
        k = random.randint(0,len(x)-limit)
        x = x[k:k+limit]
    return x
    
#chcem generovat batch data tak by som tam mal od rovnakeho pouzivatele rozne dve slova a nic ine.

def gen_data(data,metadata,batch_size,fold=0,limit=4,train=True):
    while True:
        u_list={}

        # robim to lokalne po batchoch 
        while len(u_list)<batch_size:
            if train:
                idx = random.choice(metadata[metadata['fold'] != fold].index.tolist())
            else: 
                idx = random.choice(metadata[metadata['fold'] == fold].index.tolist())
            
            # vyberiem nahodne prvok a budujem zoznam
            w,u = metadata.loc[idx,['word','user_id']]
            
            # ak sa tam nenachadza zaradim ho 
            if u_list.get(u,'xxx')=='xxx':
                u_list[u] = w
                k = 0

                yield sub_vec(data[idx],limit=limit),0
                
                # najdem k nemu odlisny par ak ho najdem vyskocim z cyklu 
                while k<1:
                    idy = random.choice(metadata[metadata['user_id'] == u].index.tolist())
                    w2 = metadata.loc[idy,'word']
                    if w2 != w:
#                         print(idy)
                        k += 1
                        yield sub_vec(data[idy],limit=limit),0



In [ ]:
w,u=metadata.loc[124,['word','user_id']]

In [ ]:
class Diar_Model(tf.keras.Model):
    def __init__(self, embedding_dim, rnn_units):
        super().__init__(self)
        self.embeding = tf.keras.layers.Dense(embedding_dim,activation='relu')
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(embedding_dim)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embeding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
batch_size=32
data_train = tf.data.Dataset.from_generator(lambda: gen_data(data,m_data,batch_size,fold=0,train=True), 
                                            output_types=(tf.float32,tf.float32), 
                                            output_shapes=((4,20),())
                                           )

data_train = data_train.batch(batch_size)
data_train.prefetch(4)

data_test = tf.data.Dataset.from_generator(lambda: gen_data(data,m_data,batch_size,fold=0,train=False), 
                                            output_types=(tf.float32,tf.float32), 
                                            output_shapes=((4,20),())
                                           )

data_test = data_test.batch(batch_size)
data_test.prefetch(4)

In [ ]:
batch_size=32
model = Diar_Model(embedding_dim=32,rnn_units=128)
for input_example_batch,true_example_batch in data_train.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
model.summary()

In [ ]:
%load_ext autoreload
%load_ext tensorboard

In [ ]:
%autoreload 2
from loss_metric import contrast_loss
model.compile(optimizer='adam', loss=contrast_loss)
steps=1000
v_steps=100
model.fit(data_train,epochs=5,validation_data = data_test,steps_per_epoch = steps,validation_steps = v_steps)

In [ ]:
model.fit(data_train,epochs=5,validation_data = data_test,steps_per_epoch = steps,validation_steps = v_steps)

In [ ]:
%autoreload 2
from loss_metric import contrast_loss
import tensorflow as tf
x = tf.random.uniform([32,6,20], minval=0, maxval=1)
contrast_loss(x,x)


In [ ]:
class DiarStep(tf.keras.Model):
    def __init__(self, model, temperature=1.0):
        super().__init__()
        self.model = model

    def generate_one_step(self, inputs, states=[],pos=0,thc=0.9):
        return_states=states.copy()
        is_old = False

        if len(states)>0:
#             print(tf.shape(states))
            st_old = states[pos]
            predict_s, st = self.model(inputs=inputs, states=st_old,return_state=True)

            st_old = st_old/tf.norm(st_old,axis=-1,keepdims=True)
            pr = predict_s/tf.norm(predict_s,axis=-1,keepdims=True)
            if tf.reduce_max(tf.tensordot(pr,st_old,axes=[0,0]))>thc:
                return_pos = pos
                return_states[pos] = st
                is_old = True
            
            else:
                for return_pos,st_old in enumerate(states):
                    predict_s, st = self.model(inputs=inputs, states=st_old,return_state=True)
                    pr = predict_s/tf.norm(predict_s,axis =-1,keepdims=True)
                    st_old = st_old/tf.norm(st_old,axis =-1,keepdims=True)
                    if tf.reduce_max(tf.tensordot(pr,st_old,axes=[0,0]))>thc:
                        return_pos = pos
                        return_states[pos] = st
                        is_old = True
                        break
            if not is_old:
                predict_s, st = self.model(inputs=inputs, states=None,return_state=True)
                return_states.append(st)
                return_pos = len(return_states)-1
                

        # este nemam ziadneho speakra teda prveho priradim do zoznamu stavov ... rozsirujem zoznam teda f je true
        else:
            predict_s, st = self.model(inputs=inputs, states=None,return_state=True)
            return_states.append(st)
            return_pos = len(return_states)-1

        # vrat predikovanu reprezentaci, zoznam stavov, a poziciu
        return predict_s, return_states, return_pos


diar_step_model = DiarStep(model)

In [ ]:
data_valid = tf.data.Dataset.from_generator(lambda: gen_data(data,m_data,batch_size,fold=0,limit=1,train=False), 
                                            output_types=(tf.float32,tf.float32), 
                                            output_shapes=((1,20),())
                                           )
data_valid.take(8)

In [ ]:
result=[]
states=[]
pos=0
maxpos=0
for s,pos_s in data_valid.take(64):
    s=tf.expand_dims(s,axis=1)
    rep, states,pos = diar_step_model.generate_one_step(s, states=states,pos=pos,thc=0.9)
    result.append(pos)


In [ ]:
result

In [ ]:
a=tf.constant([[[1,2,3,4],[1.1,2.1,3.1,4],[1.2,1.8,3,4]],[[2,1,4,3],[2.1,1.1,4.1,3],[2.2,0.8,4,3.1]]],dtype=tf.float32)

In [ ]:
tf.shape(a)

In [ ]:
def make_rep(inplen, channels, kernel_regularizer,initializer,constraint,batch_size=32,drp_rate=0.1,train=True):
    # vstupna vrstva
    input_layer = Input(shape=(inplen, channels),batch_size=batch_size, dtype = 'float32', name='input')
    x = input_layer
#     x = LayerNormalization(name = 'LN_0_'+str(i))(x)
    y = x
    x = Conv1D(filters=20, kernel_size=inplen-1, activation='relu', strides=1, 
                  kernel_constraint=constraint, 
                  kernel_initializer=initializer, 
                  kernel_regularizer=kernel_regularizer, padding='valid', trainable=train, name='conv_0_'+str(i))(x)
    x = Conv1D(filters=20, kernel_size=1, activation='relu', strides=1, 
                  kernel_constraint=constraint, 
                  kernel_initializer=initializer, 
                  kernel_regularizer=kernel_regularizer, padding='valid', trainable=train, name='conv_0_'+str(i))(x)
    x = GRU
    x = Lambda(lambda tt: tf.reshape(tt,[-1,20]))(x)
    x = LayerNormalization(axis=-1, center=False, scale=False)(x) 
    x = Lambda(lambda tt: tf.tensordot(tt,tt,axes=[1,1]))(x) 

In [ ]:
generator=gen_data(data,m_data,4)

next(generator)

In [ ]:
tf.shape(a)

In [ ]:
labels

In [ ]:
import librosa
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm

DATA_PATH = "./data/"


# Input: Folder Path
# Output: Tuple (Label, Indices of the labels, one-hot encoded labels)
def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)


# Handy function to convert wav2mfcc
def wav2mfcc(file_path, max_len=11):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=16000)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc


def save_data_to_array(path=DATA_PATH, max_len=11):
    labels, _, _ = get_labels(path)

    for label in labels:
        # Init mfcc vectors
        mfcc_vectors = []

        wavfiles = [path + label + '/' + wavfile for wavfile in os.listdir(path + '/' + label)]
        for wavfile in tqdm(wavfiles, "Saving vectors of label - '{}'".format(label)):
            mfcc = wav2mfcc(wavfile, max_len=max_len)
            mfcc_vectors.append(mfcc)
        np.save(label + '.npy', mfcc_vectors)


def get_train_test(split_ratio=0.6, random_state=42):
    # Get available labels
    labels, indices, _ = get_labels(DATA_PATH)

    # Getting first arrays
    X = np.load(labels[0] + '.npy')
    y = np.zeros(X.shape[0])

    # Append all of the dataset into one single array, same goes for y
    for i, label in enumerate(labels[1:]):
        x = np.load(label + '.npy')
        X = np.vstack((X, x))
        y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

    assert X.shape[0] == len(y)

    return train_test_split(X, y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)



def prepare_dataset(path=DATA_PATH,metadata):
    vectors = []
    for i in metadata.index.tolist():
        fldr = metadata.loc[i,'word']
        file_name = metadata.loc[i,'file']
        wave, sr = librosa.load(path+wavfile+'/'+file_name, mono=True, sr=None)
        # Downsampling
        wave = wave[::3]
        mfcc = librosa.feature.mfcc(wave, sr=16000)
        vectors.append(mfcc)

    metadata['mfcc'] = vectors

    return metadata


def load_dataset(path=DATA_PATH):
    data = prepare_dataset(path)

    dataset = []

    for key in data:
        for mfcc in data[key]['mfcc']:
            dataset.append((key, mfcc))

    return dataset[:100]

